## Redispatch data of wind power plants in Langenhorn (Schleswig-Holstein)

In [79]:
# packages
import pandas as pd
from datetime import datetime

**Redipatch data of the location Langenhorn (not only wind)**

Website Redispatch data SH-Netz (DSO): https://www.sh-netz.com/de/energie-einspeisen/redispatch-2-0/einspeisemanagement/veroeffentlichungen/abgeschlossene-massnahmen.html

In [80]:
df_redispatch = pd.read_csv("../data/Redispatch_Langenhorn.csv", sep = ';')

columns_to_keep = ['Start', 'Ende', 'Dauer (Min)', 
                   'Stufe (%)', 'Ursache', 'Anlagenschlüssel'] 
                   #'Ort Engpass', 'Einsatz-ID', , 'Gebiet', 'Anforderer', 'Netzbetreiber',
                   #'Anlagen-ID', 'Entschädigungspflicht']

column_name_mapping = {
#    'Einsatz-ID': 'mission_ID',
    'Start': 'start_redispatch',
    'Ende': 'end_redispatch',
    'Dauer (Min)': 'duration (min)',
#    'Gebiet': 'location',
#    'Ort Engpass': 'congestion_location',
    'Stufe (%)': 'level',
    'Ursache': 'cause',
    'Anlagenschlüssel': 'plant_key_eeg',
#    'Anforderer': 'requester',
#    'Netzbetreiber': 'grid_operator',
#    'Anlagen-ID': 'plant_ID',
#    'Entschädigungspflicht': 'compensation_obligation'
}
df_redispatch = df_redispatch[columns_to_keep].rename(columns=column_name_mapping)

translation_dict = {
    'Netzengpass': 'grid congestion',
    'Funktionsnachweis': 'functional proof',
    'Kundenfunktionstest': 'customer function test',
    'Test': 'test',
    'Direktvermarkter': 'direct marketer',
    'Sonstige': 'other', 
    'Vorgelagerter Netzbetreiber': 'upstream grid operator'
    }

df_redispatch['cause'] = df_redispatch['cause'].map(translation_dict).fillna(df_redispatch['cause'])

# change data types of columns 
df_redispatch['start_redispatch'] = pd.to_datetime(df_redispatch['start_redispatch'], errors='coerce', format='mixed', dayfirst=True)
df_redispatch['end_redispatch'] = pd.to_datetime(df_redispatch['end_redispatch'], errors='coerce', format='mixed', dayfirst=True)

**EEG-keys and plant type data of Langenhorn (not only wind)**

Website:
http://www.energymap.info/energieregionen/DE/105/119/477/19932.html

In [81]:
df_eeg = pd.read_csv("../data/EEG_Langenhorn.csv", sep = ';')

columns_to_keep = ['Anlagenschluessel', 'Anlagentyp', 'Inbetriebnahme', 'DSO', 'TSO', 
                   #'Nennleistung(kWp_el)', 'kWh(2013)', 'kWh(average)', 'kWh/kW', 
                   'GPS-Lat', 'GPS-Lon']

column_name_mapping = {
    'Inbetriebnahme': 'commissioning_date',
    'Anlagenschluessel': 'plant_key_eeg',
    'Anlagentyp': 'plant_type',
#    'Nennleistung(kWp_el)': 'rated_capacity_kWp', # bruttoleistung
    'DSO': 'dso',
    'TSO': 'tso',
#    'kWh(2013)': 'kWh_2013',
#    'kWh(average)': 'avg_kWh',
#    'kWh/kW': 'kWh/kW',
    'GPS-Lat': 'lat',
    'GPS-Lon': 'long'
}

df_eeg = df_eeg[columns_to_keep].rename(columns=column_name_mapping)

# change data types of columns 
df_eeg['commissioning_date'] = pd.to_datetime(df_eeg['commissioning_date'], errors='coerce', format='mixed', dayfirst=True)
#df_eeg['rated_capacity_kWp'] = df_eeg['rated_capacity_kWp'].str.replace(',', '.')
#df_eeg['kWh_2013'] = df_eeg['kWh_2013'].str.replace('.', '')
#df_eeg['avg_kWh'] = df_eeg['avg_kWh'].str.replace('.', '')
#df_eeg['avg_kWh'] = df_eeg['avg_kWh'].str.replace(',', '.')
df_eeg['lat'] = df_eeg['lat'].str.replace(',', '.')
df_eeg['long'] = df_eeg['long'].str.replace(',', '.')
#df_eeg['kWh_2013'] = df_eeg['kWh_2013'].astype(float)
#df_eeg['rated_capacity_kWp'] = df_eeg['rated_capacity_kWp'].astype(float)
#df_eeg['avg_kWh'] = df_eeg['avg_kWh'].astype(float)
df_eeg['lat'] = df_eeg['lat'].astype(float)
df_eeg['long'] = df_eeg['long'].astype(float)

translation_dict = {
    'Solarstrom': 'solar',
    'Biomasse': 'biomass',
    'Windkraft': 'wind',
    }

df_eeg['plant_type'] = df_eeg['plant_type'].map(translation_dict).fillna(df_eeg['plant_type'])

df_eeg.dropna(inplace = True)

**Filter wind power plants in df_redispatch by mapping EEG-keys of wind plants from df_eeg**

In [82]:
df_redispatch_wind = pd.merge(df_redispatch, df_eeg[df_eeg['plant_type'] == 'wind'], on='plant_key_eeg', how='inner')


In [83]:
df_redispatch_wind.columns

Index(['start_redispatch', 'end_redispatch', 'duration (min)', 'level',
       'cause', 'plant_key_eeg', 'plant_type', 'commissioning_date', 'dso',
       'tso', 'lat', 'long'],
      dtype='object')

In [84]:
df_redispatch_wind['plant_key_eeg'].unique() # 12 unique wind plants 

array(['E20793012S12X00000000002414080001',
       'E20793012S12Z00000000002414080001',
       'E20793012S12000000000002414080001',
       'E20793012S12Y00000000002414080001',
       'E20793012S12V00000000002414080001',
       'E20793012S12W00000000002414080001',
       'E2079301EA01000000000070577400001',
       'E2079301EA01000000000070577400002',
       'E2079301EA01000000000070577400003',
       'E2079301EA01000000000070577400004',
       'E2079301EA01000000000070577400005',
       'E2079301EA01000000000070577400006'], dtype=object)

**Look up if the 12 identified wind plants are registered in the Marktstammdatenregister**

Website:
https://www.marktstammdatenregister.de/MaStR/Einheit/Einheiten/OeffentlicheEinheitenuebersicht

In [85]:
# Marktstammdatenregister Windkraft Langenhorn
df_register = pd.read_csv("../data/Marktstammdatenregister_Langenhorn.csv", sep = ';')

# select relevant features
columns_to_keep = ['Anzeige-Name der Einheit', 'MaStR-Nr. der Einheit', 'Inbetriebnahmedatum der Einheit',
                   #'Bruttoleistung der Einheit', 'Nettonennleistung der Einheit', '\tMaStR-Nr. des Anlagenbetreibers',
                   'Name des Anlagenbetreibers (nur Org.)']

# translate column names
column_name_mapping = {
    'Anzeige-Name der Einheit': 'unit_name',
    'MaStR-Nr. der Einheit': 'plant_key_mastr',
    'Inbetriebnahmedatum der Einheit': 'commissioning_date',
#    'Bruttoleistung der Einheit': 'gross_capacity',
#    'Nettonennleistung der Einheit': 'net_capacity',
#    '\tMaStR-Nr. des Anlagenbetreibers': 'operator_key',
    'Name des Anlagenbetreibers (nur Org.)': 'operator_name'
}

df_register = df_register[columns_to_keep].rename(columns=column_name_mapping)
df_register['commissioning_date'] = pd.to_datetime(df_register['commissioning_date'], errors='coerce', format='mixed', dayfirst=True)

In [86]:
# Merge the DataFrames on 'commissioning_date'
df_redispatch_wind_registered = pd.merge(df_redispatch_wind, df_register, on='commissioning_date', how='inner')

In [87]:
# manually looked up latitude and longitude values for each plant_key_mastr
coordinates = {
    'SEE987798529121': {'lat': '54.6727', 'long': '8.87871'},
    'SEE929976813660': {'lat': '54.67238', 'long': '8.86128'},
    'SEE955793159025': {'lat': '54.674777', 'long': '8.885012'},
    'SEE975321353732': {'lat': '54.67212', 'long': '8.86985'},
    'SEE964462351561': {'lat': '54.6594', 'long': '8.85654'},
    'SEE989327012883': {'lat': '54.667118', 'long': '8.862491'},
    'SEE944503500066': {'lat': '54.664171', 'long': '8.86336'},
    'SEE967713858420': {'lat': '54.666857', 'long': '8.869743'},
    'SEE978968638712': {'lat': '54.667329', 'long': '8.87676'},
    'SEE968633677399': {'lat': '54.664481', 'long': '8.877575'},
    'SEE923447900071': {'lat': '54.667751', 'long': '8.883703'}
}

for index, row in df_redispatch_wind_registered.iterrows():
    plant_key = row['plant_key_mastr']
    lat = coordinates[plant_key]['lat']
    long = coordinates[plant_key]['long']
    df_redispatch_wind_registered.at[index, 'lat'] = lat
    df_redispatch_wind_registered.at[index, 'long'] = long

In [88]:
unique_plants = df_redispatch_wind_registered[['plant_key_eeg', 'plant_key_mastr', 'lat', 'long', 'operator_name']].drop_duplicates().reset_index(drop = True)

In [89]:
# store csv
unique_plants.to_csv('wind_plants_Langenhorn.csv',sep = ';', index=False)

**Create a df minute-by-minute redispatch of the 11 wind plants found in the register**

In [90]:
# remove duplicates and unnecessary columns
df_redispatch_wind_registered = df_redispatch_wind_registered.drop_duplicates()
df_redispatch_wind_registered.reset_index(inplace=True, drop=True)

columns_to_remove = ['duration (min)', 'commissioning_date']
df_redispatch_wind_registered.drop(columns_to_remove, axis = 1, inplace = True)

In [91]:
# last two years
# latest_end = df_redispatch_wind_registered['end_redispatch'].max()
# earliest_start = latest_end - timedelta(days=365 * 2)

# one year (2023)
earliest_start = datetime(2023, 1, 1, 0, 0, 0)
latest_end = datetime(2024, 1, 1, 0, 0, 0)

time_index = pd.date_range(start=earliest_start, end=latest_end, freq='10T') # all 10 min
subset_df = df_redispatch_wind_registered[(df_redispatch_wind_registered['start_redispatch'] >= earliest_start) & (df_redispatch_wind_registered['end_redispatch'] <= latest_end)]

In [92]:
wind_redispatch = pd.DataFrame(index=time_index)
wind_redispatch['redispatch'] = 0
columns_to_copy = ['level', 'cause', 'plant_key_eeg', 'plant_key_mastr',
                   'lat', 'long', 'operator_name', 'dso', 'tso']
for column in columns_to_copy:
    wind_redispatch[column] = ''

# iterate over the subset_df    
for _, row in subset_df.iterrows():
    start = row['start_redispatch']
    end = row['end_redispatch']
    mask = (wind_redispatch.index >= start) & (wind_redispatch.index <= end)
    wind_redispatch.loc[mask, 'redispatch'] = 1
    for column in columns_to_copy:
        wind_redispatch.loc[mask, column] = row[column]

# replace empty entries when redispatch is 0 with 0 instead of ""
wind_redispatch.loc[wind_redispatch['redispatch'] == 0, :] = 0     

In [93]:
wind_redispatch[wind_redispatch["redispatch"] == 1]

,redispatch,level,cause,plant_key_eeg,plant_key_mastr,lat,long,operator_name,dso,tso
2023-01-03 23:50:00,1,0,grid congestion,E20793012S12V00000000002414080001,SEE964462351561,54.6594,8.85654,Ockholm- Langenhorn Erlös- und Infrastruktur G...,Schleswig-Holstein Netz AG (E.ON Hanse AG),TenneT TSO GmbH
2023-01-04 00:00:00,1,0,grid congestion,E20793012S12V00000000002414080001,SEE964462351561,54.6594,8.85654,Ockholm- Langenhorn Erlös- und Infrastruktur G...,Schleswig-Holstein Netz AG (E.ON Hanse AG),TenneT TSO GmbH
2023-01-04 00:10:00,1,0,grid congestion,E20793012S12V00000000002414080001,SEE964462351561,54.6594,8.85654,Ockholm- Langenhorn Erlös- und Infrastruktur G...,Schleswig-Holstein Netz AG (E.ON Hanse AG),TenneT TSO GmbH
2023-01-04 00:20:00,1,0,grid congestion,E20793012S12V00000000002414080001,SEE964462351561,54.6594,8.85654,Ockholm- Langenhorn Erlös- und Infrastruktur G...,Schleswig-Holstein Netz AG (E.ON Hanse AG),TenneT TSO GmbH
2023-01-04 00:30:00,1,0,grid congestion,E20793012S12V00000000002414080001,SEE964462351561,54.6594,8.85654,Ockholm- Langenhorn Erlös- und Infrastruktur G...,Schleswig-Holstein Netz AG (E.ON Hanse AG),TenneT TSO GmbH
...,...,...,...,...,...,...,...,...,...,...
2023-12-14 16:10:00,1,0,grid congestion,E2079301EA01000000000070577400006,SEE923447900071,54.667751,8.883703,Bürgerwindpark Langenhorn II,Schleswig-Holstein Netz AG (E.ON Hanse AG),TenneT TSO GmbH
2023-12-14 16:20:00,1,0,grid congestion,E2079301EA01000000000070577400006,SEE923447900071,54.667751,8.883703,Bürgerwindpark Langenhorn II,Schleswig-Holstein Netz AG (E.ON Hanse AG),TenneT TSO GmbH
2023-12-14 16:30:00,1,0,grid congestion,E2079301EA01000000000070577400006,SEE923447900071,54.667751,8.883703,Bürgerwindpark Langenhorn II,Schleswig-Holstein Netz AG (E.ON Hanse AG),TenneT TSO GmbH
2023-12-14 16:40:00,1,0,grid congestion,E2079301EA01000000000070577400006,SEE923447900071,54.667751,8.883703,Bürgerwindpark Langenhorn II,Schleswig-Holstein Netz AG (E.ON Hanse AG),TenneT TSO GmbH


In [94]:
# store csv
wind_redispatch.to_csv('wind_redispatch_2023.csv',sep = ';', index=False)